In [ ]:
from ctraptools.kymos.detect import Detector
from lumicks import pylake

import ctraptools.kymos.stepfinding as ksf
import ctraptools.kymos.io as kio
import os

In [ ]:
# Parameters
path = "C:\\Users\\steph\\Documents\\People\\CTrapTools\\20220616-150818 Kymograph 7.h5"
channel = 0

In [ ]:
# Loading kymograph as imagee
file = pylake.File(path)
kymo = next(iter(file.kymos.values()))
image = kymo.get_image()[:,:,channel]

rootpath = os.path.dirname(path)+'/'
rootname = os.path.basename(os.path.splitext(path)[0])

In [ ]:
# Initialising Detector
detector = Detector(half_t_w=5,peak_det_thresh = 2, n_max=10, c_ub=5, c_def=3, min_track_length = 10)

In [ ]:
# (Optional) Testing fit parameters
fr = 161
peaks = detector.fit_peaks(image,fr)
kio.plot_gauss_for_frame(peaks,fr,image)

In [ ]:
# Detect peaks and track
tracks = detector.detect(image)

In [ ]:
# Measuring intensity
for track in tracks.values():
    track.measure_intensity(image)

# Step fitting (to be added later)
# ksf.fit_steps(tracks,image)

# Saving files
kio.save_overlay(tracks,image,rootpath+rootname+'_C'+str(channel)+"_overlay")
kio.write_traces(tracks,rootpath+rootname+'_C'+str(channel)+"_traces")
kio.save_plots(tracks,rootpath+rootname+'_C'+str(channel)+"_traces")
# kio.write_change_points(tracks,rootpath+rootname+'_C'+str(channel)+'_steps')